# Chatbot

## Download and import library

In [1]:
!pip install malaya

In [7]:
!pip install pytorch

  Running setup.py clean for pytorch
Failed to build pytorch
    Running setup.py install for pytorch: started
    Running setup.py install for pytorch: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\User\anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\User\\AppData\\Local\\Temp\\pip-install-r4oxm51u\\pytorch_b043495d5aeb4751acf416b864b25966\\setup.py'"'"'; __file__='"'"'C:\\Users\\User\\AppData\\Local\\Temp\\pip-install-r4oxm51u\\pytorch_b043495d5aeb4751acf416b864b25966\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\User\AppData\Local\Temp\pip-wheel-zvwexhl1'
       cwd: C:\Users\User\AppData\Local\Temp\pip-install-r4oxm51u\pytorch_b043495d5aeb4751acf416b864b25966\
  Complete output (5 lines):
  Traceback (most recent call last):
    File "<string>", line 1, in <module>
    File "C:\Users\User\AppData\Local\Temp\pip-install-r4oxm51u\pytorch_b043495d5aeb4751acf416b864b25966\setup.py", line 15, in <module>
      raise

In [10]:
!pip install PySastrawi

  Using cached PySastrawi-1.2.0-py2.py3-none-any.whl (210 kB)


In [25]:
!pip install Spylls

In [1]:
#!pip install nltk
#!pip install keras
#!pip install tensorflow
#!pip install speechrecognition
#!pip install -U nltk

In [1]:
import malaya
import nltk
import pickle
import numpy as np
import json
import random
import speech_recognition as sr

C:\Users\User\anaconda3\lib\site-packages\malaya_boilerplate\frozen_graph.py:35: UserWarning: Cannot import beam_search_ops from Tensorflow Addons, ['malaya.jawi_rumi.deep_model', 'malaya.phoneme.deep_model', 'malaya.rumi_jawi.deep_model', 'malaya.stem.deep_model'] will not available to use, make sure Tensorflow Addons version >= 0.12.0
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\malaya_boilerplate\frozen_graph.py:38: UserWarning: check compatible Tensorflow version with Tensorflow Addons at https://github.com/tensorflow/addons/releases
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\malaya\tokenizer.py:202: FutureWarning: Possible nested set at position 3361
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
C:\Users\User\anaconda3\lib\site-packages\malaya\tokenizer.py:202: FutureWarning: Possible nested set at position 3879
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [2]:
from nltk.stem import WordNetLemmatizer
from keras.models import load_model

In [3]:
# lemmatizer = WordNetLemmatizer()
# for malay
sastrawi = malaya.stem.sastrawi()
model = load_model('malay_model_test.h5')
intents = json.loads(open('intents3.json').read())
words = pickle.load(open('words2.pkl','rb'))
labels = pickle.load(open('labels2.pkl','rb'))
# stopwords = json.loads(open('malay_stopwords.json').read())

## Preprocessing Data

In [4]:
def clean_up_sentence(sentence):
#     spell check
    spelling_model = malaya.spelling_correction.spylls.load()
    sentence_words = spelling_model.correct_text(sentence)
    #tokenize the pattern
    tokenizer = malaya.tokenizer.Tokenizer()
    sentence_words = tokenizer.tokenize(sentence_words)
    
#     sentence_words = nltk.word_tokenize(sentence)
    #lemmatize - create short form
    sentence_words = [sastrawi.stem(word.lower()) for word in sentence_words]
#     print(sentence_words)
    return sentence_words

## Bag of words array : 0 to 1 for each word in the bag that exists

In [5]:
def bow(sentence, words, show_details= True):
    # tokenize pattern
    sentence_words = clean_up_sentence(sentence)
    # bow : matrix of N words, vocab matrix
    bag = [0]* len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocab position
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    
    return(np.array(bag))

## Speech to text

In [6]:
def stt():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        #remove ambient noise
        r.adjust_for_ambient_noise(source)
        # read the audio data from the default microphone
        audio_data = r.listen(source)
        # print("Recognizing...")
        # convert speech to text
        text = r.recognize_google(audio_data, language="ms-MY")
        print(f"\nAnda: {text}\n")
        return text
        # print(text)

## Response

In [10]:
def predict_label():
#     default tag list for tracking user's intents
    tag_list=['default']
#     initiating the chatbot
#     welcoming message
    print("\nHi, selamat datang ke Farmasi NLP!\n1. Taip 'mic'--untuk guna mic\n2. Masukkan simptom\n3. Pertanyaan tentang pencegahan penyakit\n4. Taip 'balik' untuk mula semula\n5. Taip 'keluar'--tamat chatbot\n")
    while True:
#       user's input
        inp = input("Anda: ")
#       exit function
        if inp.lower()=="keluar":
            print("\nMediBot: Jumpa lagi\n")
            break
#       speech to text initiating microphone
        elif inp.lower() == "mic":
            print("\nMediBot: Teruskan, aku dengar dengan jelas\n")
            inp = stt()
            if inp.lower() =='keluar':
                print("\nMediBot: Jumpa lagi\n")
                break
#       predicting labels 'tag'
        results = model.predict(np.array([bow(inp, words, show_details= False)]))[0]
        results_index = np.argmax(results) 
        tag = labels[results_index]
        tag_list.append(tag)  
#         print(results[results_index])
        if len(tag_list) > 2:
            tag_list=tag_list[2:]
#       follow-up asking function
        if inp.lower() in ['y']:
#             print(tag)
#             print(tag_list)
            if tag_list == ['default','error']:
                results[results_index] =0.0
                tag_list =['default', 'error']
            else:
                tag = tag_list[0].replace('symptoms','prevention')
                results[results_index] =0.8
                tag_list = ['default', tag]
#                 print(tag)
#                 print(tag_list)
        elif inp.lower() in ['t']:
            if tag_list == ['default','error']:
                results[results_index] =0.0
                tag_list =['default', 'error']
            else:
                tag = 'goodbye'
                results[results_index] =0.8
                tag_list = ['default', tag]
#       return to menu function      
        elif inp.lower() == "balik":
            tag = 'return'
            results[results_index] =0.8
            tag_list = ['default']

            
#       accuracy threshold    
        if results[results_index] > 0.7:
            for tg in intents['intents']:
                if tg['tag'] == tag:
                    responses = tg['responses']
                    tag_list.append(tag)
            print(f"\nMediBot: {random.choice(responses)}\n")

        else:
#           error detection
            print(f"\nMediBot: Maaf, saya tidak faham. adakah anda bermaksud untuk mengatakan {clean_up_sentence(inp.lower())}? sila taip 'y' untuk ya atau 't' untuk tidak\n")
#             print(tag_list)
#           initiating input and mic
            inp = input("Anda: ")
            if inp.lower()=="keluar":
                print("\nMediBot: Jumpa lagi\n")
                break
            elif inp.lower() == "mic":
                print("\nMediBot: Teruskan, aku dengar dengan jelas\n")
                inp = stt()
                if inp.lower() =='keluar':
                    print("\nMediBot: Jumpa lagi\n")
                    break
            elif inp.lower() in ['y']:
#                 print(tag)
                for tg in intents['intents']:
                    if tg['tag'] == tag:
                        responses = tg['responses']
                print(f"\nMediBot: {random.choice(responses)}\n")
                tag=tag_list[1]
                tag_list.append(tag)
                
            elif inp.lower() in ['t']:
                tag_list=['default']
                print("\nHi, selamat datang ke Farmasi NLP!\n1. Taip 'mic'--untuk guna mic\n2. Masukkan simptom\n3. Pertanyaan tentang pencegahan penyakit\n4. Taip 'balik' untuk mula semula\n5. Taip 'keluar'--tamat chatbot\n")
            else:
                tag='error'
                for tg in intents['intents']:
                    if tg['tag'] == tag:
                        responses = tg['responses']
                        tag_list= ['default']
                print(f"\nMediBot: {random.choice(responses)}\n")
                tag_list=['default']

# Chatbot deployment

In [12]:
predict_label()


Hi, selamat datang ke Farmasi NLP!
1. Taip 'mic'--untuk guna mic
2. Masukkan simptom
3. Pertanyaan tentang pencegahan penyakit
4. Taip 'balik' untuk mula semula
5. Taip 'keluar'--tamat chatbot

Anda: sya xde miant
1/1 [==============================] - 0s 122ms/step

MediBot: Maaf, saya tidak faham. adakah anda bermaksud untuk mengatakan ['saya', 'tidak', 'ada', 'minat']? sila taip 'y' untuk ya atau 't' untuk tidak

Anda: y

MediBot: Nampaknya anda mengalami kemurungan
          Untuk mengetahui lebih info, sila taip 'y' atau 't' untuk tidak

Anda: y
1/1 [==============================] - 0s 17ms/step

MediBot: (a) Ubat-ubatan yang boleh anda makan: 1) Brexpiprazole 2) Quetiapine 3) Olanzapine. (b) Pencegahan yang mesti anda ikuti: 1) Bersenam secara teratur 2) Potong masa media sosial 3) Minum air sebagai minuman utama anda 4) Membina hubungan yang kuat 5) Meminimumkan pilihan harian anda 6) Ikuti karbohidrat yang sangat rendah diet. (c) Makanan yang dicadangkan adalah: 1) Dapatkan c